gemaakt voor LIWO door [David Haasnoot](d.haasnoot@hkv.nl)

In [ ]:
import dotenv
import pandas as pd

In [ ]:
from update_local_bulk_LDO import  haal_token_op
from export_LDO import list_bulk_export, delete_bulk_export, delete_bulk_export_errors, archive_bulk_export

Zie `readme.md` voor uitgebreid stappen plan voor het aanmaken van een api key.

In het kort
- Login & ga naar https://www.ldo.overstromingsinformatie.nl/auth/
- Onder `V1`, `POST auth/v1/personalapikeys`, klik op `try it out` en vervang body met:
```json
{
  "scope": "*:readwrite",
  "name": "personalAPIkeyLIWOexport...",
  "expiry_date": "2029-12-31T23:59:59.037Z",
  "revoked": false
}
```
- Pas de datum en name aan waar nodig, klik op `Excecute` in het blauw. In de response body staat: 
```json
{
...
  "key": "xxxxxx.yyyyyyyyyyyyyyy",
...
}
```
- Bewaar die hele `'key'` in een bestand die `.env` heet, zie `.env.example` voor het formaat.

In [ ]:
# haal de API key op uit de .env file
if dotenv.load_dotenv():
    environmental_variables = dotenv.dotenv_values()
    LDO_api_key = environmental_variables["LDO_api_key"]\
# Of zet hier de API key handmatig in
# LDO_api_key = "abcd"
TENANT : int  = 1 # 0, 1, 2 ...headers = haal_token_op(LDO_api_key, tenant=TENANT)

In [ ]:
data, response = list_bulk_export(headers)

In [ ]:
df = pd.DataFrame(data)
assert len(df) == response.json()['total']
df['created_by_name'] = df['created_by'].apply(lambda x: x['name'] if isinstance(x, dict) else x)
# df['created_by_name'].unique(),
df['status'].unique()

In [ ]:
import requests
server = "https://www.ldo.overstromingsinformatie.nl"


## delete errors:

In [ ]:
df_user = df[df['created_by_name'] == 'D.Haasnoot@hkv.nl']
df_status_error = df_user[df_user['status'] == 'error']

In [ ]:
ids_to_delete = df_status_error["id"].to_list()

In [ ]:
ids_to_delete

Exporteert de gewenste bestanden (kan lang duren)

In [ ]:
lst = []
for id in ids_to_delete:
    success_error, response_error = delete_bulk_export_errors(
                                        index=id,
                                        headers=headers,
    )
    success_delete, response_delete = delete_bulk_export(
                                            index=id,
                                            headers=headers,
    )
    lst.append((id, success_error, response_error, success_delete, response_delete))


## Archive:

In [ ]:
df_status_complete = df_user[df_user['status'] == "complete"]
# df_status_complete = df_user[df_user['status'] == "submitted"]

In [ ]:
ids_to_archive = df_status_complete["id"].to_list()

In [ ]:
ids_to_archive

In [ ]:
for index in ids_to_archive: 
    success, response = archive_bulk_export(headers=headers, index=index)